In [267]:
import pandas as pd
from fredapi import Fred
import sys
import os
import hvplot.pandas as hvplot
# Add config module to sys path
sys.path.insert(0, os.path.abspath('../../config'))

# Import API key
from api_keys import fred_api_key


In [268]:
#Set up API Key

fred = Fred(api_key=f'{fred_api_key}')

In [269]:
#Retrieve 10 Year Treasury security rates (monthly) from FRED API

data_10year_treasury = fred.get_series('GS10')

In [270]:
# Retain only data since June 1976. This is to match with data available for 2 year Treasury securities
data_10year_treasury = data_10year_treasury.loc[data_10year_treasury.index > '1958-12-31']

In [271]:
data_10year_treasury.head(5)

1959-01-01    4.02
1959-02-01    3.96
1959-03-01    3.99
1959-04-01    4.12
1959-05-01    4.31
dtype: float64

In [272]:
data_2year_treasury = fred.get_series('GS2')

In [273]:
data_2year_treasury.head()

1976-06-01    7.06
1976-07-01    6.85
1976-08-01    6.63
1976-09-01    6.42
1976-10-01    5.98
dtype: float64

In [274]:
data_1year_treasury = fred.get_series('GS1')

In [275]:
data_1year_treasury = data_1year_treasury.loc[data_1year_treasury.index > '1958-12-31']

In [276]:
data_1year_treasury.head()

1959-01-01    3.36
1959-02-01    3.54
1959-03-01    3.61
1959-04-01    3.72
1959-05-01    3.96
dtype: float64

In [277]:
data_10year_treasury_df = pd.DataFrame(data_10year_treasury)

data_2year_treasury_df = pd.DataFrame(data_2year_treasury)

data_1year_treasury_df = pd.DataFrame(data_1year_treasury)


In [278]:
data_10year_treasury_df = data_10year_treasury_df.reset_index()

In [279]:
data_10year_treasury_df.head()

,index,0
0,1959-01-01,4.02
1,1959-02-01,3.96
2,1959-03-01,3.99
3,1959-04-01,4.12
4,1959-05-01,4.31


In [280]:
data_2year_treasury_df = data_2year_treasury_df.reset_index()

In [281]:
data_2year_treasury_df.head()

,index,0
0,1976-06-01,7.06
1,1976-07-01,6.85
2,1976-08-01,6.63
3,1976-09-01,6.42
4,1976-10-01,5.98


In [282]:
data_1year_treasury_df = data_1year_treasury_df.reset_index()

In [283]:
data_1year_treasury_df.head()

,index,0
0,1959-01-01,3.36
1,1959-02-01,3.54
2,1959-03-01,3.61
3,1959-04-01,3.72
4,1959-05-01,3.96


In [284]:
treasury_df = pd.merge(data_10year_treasury_df,data_2year_treasury_df, on="index", how="left")

In [285]:
treasury_df = treasury_df.fillna(0)

In [286]:
treasury_df.head()

,index,0_x,0_y
0,1959-01-01,4.02,0.0
1,1959-02-01,3.96,0.0
2,1959-03-01,3.99,0.0
3,1959-04-01,4.12,0.0
4,1959-05-01,4.31,0.0


In [287]:
treasury_df = pd.merge(treasury_df, data_1year_treasury_df, on="index")

In [288]:
treasury_df.head()

,index,0_x,0_y,0
0,1959-01-01,4.02,0.0,3.36
1,1959-02-01,3.96,0.0,3.54
2,1959-03-01,3.99,0.0,3.61
3,1959-04-01,4.12,0.0,3.72
4,1959-05-01,4.31,0.0,3.96


In [289]:
treasury_df = treasury_df.rename(columns={"index":"date", 
                                          "0_x":"10 Years T Rates", 
                                          "0_y": "2 Years T Rates",
                                         0 : "1 Year T Rates"})

In [290]:
treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates
0,1959-01-01,4.02,0.0,3.36
1,1959-02-01,3.96,0.0,3.54
2,1959-03-01,3.99,0.0,3.61
3,1959-04-01,4.12,0.0,3.72
4,1959-05-01,4.31,0.0,3.96


In [291]:
def adjust_2_year_missing_rates(twoYearValue, tenYearValue):
    if twoYearValue == 0:
        return tenYearValue
    else:
        return twoYearValue

In [292]:
treasury_df["2 Years T Rates"] = treasury_df.apply(lambda row :  adjust_2_year_missing_rates(row["2 Years T Rates"], row["10 Years T Rates"]), axis=1)

In [293]:
treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates
0,1959-01-01,4.02,4.02,3.36
1,1959-02-01,3.96,3.96,3.54
2,1959-03-01,3.99,3.99,3.61
3,1959-04-01,4.12,4.12,3.72
4,1959-05-01,4.31,4.31,3.96


In [294]:
treasury_df["10-2 Rate Diff"] = treasury_df["10 Years T Rates"] - treasury_df["2 Years T Rates"]

In [295]:
treasury_df["10-1 Rate Diff"] = treasury_df["10 Years T Rates"] - treasury_df["1 Year T Rates"]

In [296]:
treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates,10-2 Rate Diff,10-1 Rate Diff
0,1959-01-01,4.02,4.02,3.36,0.0,0.66
1,1959-02-01,3.96,3.96,3.54,0.0,0.42
2,1959-03-01,3.99,3.99,3.61,0.0,0.38
3,1959-04-01,4.12,4.12,3.72,0.0,0.40
4,1959-05-01,4.31,4.31,3.96,0.0,0.35


In [297]:
treasury_df.dtypes

date                datetime64[ns]
10 Years T Rates           float64
2 Years T Rates            float64
1 Year T Rates             float64
10-2 Rate Diff             float64
10-1 Rate Diff             float64
dtype: object

In [298]:
treasury_df['Year-Qtr'] = pd.PeriodIndex(treasury_df.date, freq='Q')

In [299]:
treasury_df = treasury_df.groupby(['Year-Qtr']).mean()

In [300]:
treasury_df = treasury_df.drop(columns=["date", "10 Years T Rates", "2 Years T Rates", "1 Year T Rates"])

In [301]:
treasury_df = treasury_df.reset_index()

In [302]:
treasury_df = treasury_df.rename(columns={"index" : "Year-Qtr"})

In [303]:
treasury_df["Year-Qtr"] = treasury_df["Year-Qtr"].astype("str")

In [304]:
treasury_df.head()

,Year-Qtr,10-2 Rate Diff,10-1 Rate Diff
0,1959Q1,0.0,0.486667
1,1959Q2,0.0,0.340000
2,1959Q3,0.0,-0.100000
3,1959Q4,0.0,-0.333333
4,1960Q1,0.0,-0.083333


In [305]:
print("Treasury_Rates_Base_For_ML completed. treasury_df is ready for use ")

Treasury_Rates_Base_For_ML completed. treasury_df is ready for use 
